In [ ]:
# %% [markdown]
# # 3. Health Data: CNES & Infrastructure
#
# **The Problem:** The CNES database is notoriously complex. It has hundreds of columns for specific equipment ("MRI machines", "Pediatric ICU beds"). Also, many units lack coordinates.
#
# **The Solution:**
# 1.  **Semantic Groups:** AtlasBR aggregates hundreds of columns into readable concepts (e.g., `total_leitos_internacao`).
# 2.  **Fallback Geocoding:** If a hospital has no GPS, we find it via its CEP (Postal Code).

# %%
import os
import atlasbr
from atlasbr.app.cnes import load_cnes

atlasbr.configure_logging()
MUNICIPALITY = "Curitiba, PR"

# %% [markdown]
# ## 1. Load with Geocoding
# We enable `geocode=True`. This tells AtlasBR: *"If you can't find the lat/lon, look up the CEP centroid."*

# %%
gdf_health = load_cnes(
    places=[MUNICIPALITY],
    year=2023,
    month=9,      # CNES is monthly!
    geocode=True, # <--- The magic fix for missing coordinates
    gcp_billing=os.getenv("GOOGLE_CLOUD_PROJECT")
)

print(f"🏥 Loaded {len(gdf_health)} health establishments.")

# %% [markdown]
# ## 2. Decoding Complexity
# Instead of raw codes, look at the `complexidade` and aggregated columns.
# We can easily filter for **High Complexity** units (Major Hospitals).

# %%
major_hospitals = gdf_health[
    (gdf_health["complexidade"] == "alta") | 
    (gdf_health["total_leitos_internacao"] > 50)
].copy()

print(f"Found {len(major_hospitals)} major facilities.")
display(major_hospitals[[
    "id_estabelecimento_cnes", 
    "total_leitos_internacao", 
    "total_salas_cirurgicas_obstetricas",
    "quantidade_trabalhadores_saude"
]].head())

# %% [markdown]
# ## 3. Interactive Map: Hospital Capacity
# Circle size represents the number of beds.
# This instantly reveals the "Health Hubs" of the city.

# %%
major_hospitals.explore(
    column="total_leitos_internacao",
    cmap="viridis",
    scheme="naturalbreaks",
    tooltip=[
        "id_estabelecimento_cnes", 
        "complexidade", 
        "total_leitos_internacao"
    ],
    style_kwds={
        "style_function": lambda x: {
            "radius": x["properties"]["total_leitos_internacao"] / 10 + 2
        }
    },
    tiles="CartoDB DarkMatter"
)